In [163]:
import pandas as pd
from sklearn.model_selection import train_test_split
import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score, auc
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score



In [149]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/data.csv')
a = ['Text']
X = data.Text
def remove_punctuation(X):
    no_punct=[words for words in X if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
X = X.apply(lambda x: remove_punctuation(x.lower()))

def tokenize(X):
    split = re.split("\W+",X) 
    return split
X = X.apply(lambda x: tokenize(x.lower()))

stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(X):
    text=[word for word in X if word not in stopword]
    return text
X = X.apply(lambda x: remove_stopwords(x))

ban=['xxxx']
def remove_xxxx(X):
    no_x = [words for words in X if words not in ban]
    words_wo_xxxx= no_x
    return words_wo_xxxx
X=X.apply(lambda x: remove_xxxx(x))

stemmer = PorterStemmer()
def word_stemmer(X):
  stem_text = " ".join([stemmer.stem(i) for i in X])
  return stem_text
X=X.apply(lambda x: word_stemmer(x))

label_encoder = LabelEncoder()
data['Class'] = label_encoder.fit_transform(data['Class'])
y= data.Class


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    outdat inform credit report previous disput ye...
1    account credit report mistaken date mail debt ...
2    compani refus provid verif valid debt per righ...
3    complaint regard squar two financi refer cfpb ...
4    start refin home mortgag process cash option x...
Name: Text, dtype: object

In [165]:
train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state = 43)
vectorizer = CountVectorizer()
vectorizer.fit(train_X)
vectorizer.fit(val_X)
X_train = vectorizer.transform(train_X)
X_val  = vectorizer.transform(val_X)

In [162]:
model = XGBClassifier(n_estimators=500,learning_rate=0.05, n_jobs=4,objective = 'multi:softmax',silent=1,tree_method='approx')
model.fit(X_train,train_y,early_stopping_rounds=5, 
             eval_set=[(X_val, val_y)],
             verbose=False)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=4,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=1, subsample=1, tree_method='approx', verbosity=1)

In [174]:
y_pred = model.predict(X_val)
print(accuracy_score(y_pred, val_y))
my_tags = ['0','1','2','3']

0.8918499754439476


In [175]:
print(classification_report(val_y, y_pred,target_names=my_tags))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88     13326
           1       0.95      0.95      0.95     10493
           2       0.87      0.84      0.86      9328
           3       0.88      0.85      0.86      5540

    accuracy                           0.89     38687
   macro avg       0.89      0.88      0.89     38687
weighted avg       0.89      0.89      0.89     38687

